# 1.0 Setup

In [1]:
import libs

# 2.0 Carregamento e Checagem

In [2]:
DF_RAW = pd.read_csv("./data/dataset_raw.csv")


In [3]:
np.random.seed(42)

In [3]:
DF_RAW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [4]:
DF_RAW.describe()

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000
mean,17.017729,70.231332,17.465186,70.845702,1.023359
std,8.185109,22.883647,7.335122,21.118812,0.839065
min,-30.905562,-88.366217,0.010000,0.010000,0.000000
25%,12.933284,73.170000,12.988453,73.280000,0.000000
50%,18.546947,75.898497,18.633934,76.002574,1.000000
75%,22.728163,78.044095,22.785049,78.107044,2.000000
max,30.914057,88.433452,31.054057,88.563452,3.000000


In [5]:
DF_RAW.sample(30)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
30818,0x1393,VADRES13DEL01,NaN,NaN,22.310237,73.158921,22.350237,73.198921,11-03-2022,NaN,13:00:00,conditions Stormy,High,2,Meal,motorcycle,0,No,Metropolitian,(min) 34
5707,0xcc03,DEHRES11DEL01,27,4.6,0.000000,0.000000,0.040000,0.040000,17-02-2022,15:15:00,15:20:00,conditions Windy,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 28
35373,0xabaa,JAPRES17DEL02,24,4.9,26.892312,75.806896,26.942312,75.856896,11-03-2022,17:40:00,17:45:00,conditions Fog,Medium,0,Snack,motorcycle,1,No,Metropolitian,(min) 24
12546,0xae50,INDORES09DEL01,22,4.9,22.725835,75.887648,22.835835,75.997648,08-03-2022,23:50:00,23:55:00,conditions Sandstorms,Low,0,Drinks,motorcycle,0,No,Metropolitian,(min) 22
14935,0xefe,HYDRES03DEL01,32,4.3,17.422819,78.449578,17.432819,78.459578,07-03-2022,08:30:00,08:45:00,conditions Windy,Low,0,Meal,motorcycle,1,No,Metropolitian,(min) 31
27884,0xc413,KOCRES16DEL01,33,4.9,10.003064,76.307589,10.043064,76.347589,17-02-2022,16:20:00,16:35:00,conditions Cloudy,Medium,2,Meal,scooter,0,No,Urban,(min) 15
12466,0xbd2d,MUMRES09DEL01,20,4.7,18.994049,72.825203,19.034049,72.865203,17-03-2022,15:40:00,15:45:00,conditions Windy,Medium,2,Snack,scooter,1,No,Metropolitian,(min) 28
19341,0x9229,INDORES13DEL03,37,4,22.745049,75.892471,22.775049,75.922471,03-03-2022,21:25:00,21:35:00,conditions Stormy,Jam,0,Snack,motorcycle,2,No,Metropolitian,(min) 40
12409,0xcc9e,AGRRES01DEL02,25,4.6,27.161661,78.011544,27.291661,78.141544,14-02-2022,18:35:00,18:45:00,conditions Sunny,Medium,1,Buffet,scooter,0,No,Urban,(min) 17
40183,0xcd8c,AURGRES18DEL02,26,5,19.888716,75.321461,19.938716,75.371461,17-02-2022,18:10:00,18:20:00,conditions Cloudy,Medium,2,Buffet,scooter,0,No,Urban,(min) 17


In [6]:
fig = plot_histogram(DF_RAW)
fig.show()

NameError: name 'plot_histogram' is not defined

In [ ]:
# A checagem de outliers tem como premissa de que os dados estão normalmente distribuídos, o que já foi visto pelos histograms acima que este não é o caso aqui.
check_outliers(DF_RAW)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0xc042,AGRRES010DEL01,34,4.7,-27.163303,78.057044,27.233303,78.127044,16-02-2022,23:00:00,23:05:00,conditions Sandstorms,Low,1,Drinks,scooter,0,No,Metropolitian,(min) 15
1,0xc044,AGRRES12DEL03,32,4.7,-27.165108,78.015053,27.225108,78.075053,13-02-2022,20:55:00,21:10:00,conditions Sandstorms,Jam,2,Meal,scooter,1,No,Metropolitian,(min) 31
2,0x4db,PUNERES02DEL03,NaN,NaN,-18.551440,-73.804855,18.611440,73.864855,05-03-2022,NaN,22:05:00,conditions Stormy,Jam,2,Meal,scooter,0,No,Urban,(min) 12
3,0xc003,DEHRES13DEL02,NaN,NaN,-30.366322,-78.070453,30.496322,78.200453,18-02-2022,NaN,22:30:00,conditions NaN,NaN,3,Snack,scooter,1,No,Metropolitian,(min) 20
4,0x473,MYSRES07DEL03,NaN,NaN,-12.325461,-76.632278,12.385461,76.692278,03-03-2022,NaN,22:45:00,conditions NaN,NaN,3,Buffet,scooter,NaN,No,Metropolitian,(min) 16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6892,0xaaea,RANCHIRES17DEL03,23,5,23.374878,85.335739,23.434878,85.395739,01-03-2022,17:20:00,17:25:00,conditions Windy,Medium,1,Buffet,scooter,1,No,Metropolitian,(min) 28
6893,0x6ecf,RANCHIRES010DEL03,20,5,23.354422,85.332900,23.444422,85.422900,08-03-2022,22:00:00,22:05:00,conditions Fog,Jam,1,Drinks,scooter,1,No,Metropolitian,(min) 41
6894,0x7f9f,RANCHIRES11DEL02,29,4.7,23.359194,85.325447,23.439194,85.405447,18-03-2022,23:30:00,23:45:00,conditions Windy,Low,2,Snack,motorcycle,1,No,Metropolitian,(min) 17
6895,0x1178,RANCHIRES16DEL01,35,4.2,23.371292,85.327872,23.481292,85.437872,08-03-2022,21:45:00,21:55:00,conditions Windy,Jam,2,Drinks,motorcycle,1,No,Metropolitian,(min) 33


# 3.0 Limpeza & Transansformação

Claramente algumas colunas possuem valores NaN, mas eles não estão aparecendo na checagem de NaNs. Isso pode ser porque eles são do tipo objeto ou estão armazenados como string e possuem algum formato estranho como " NaN", com um espaço em branco.<br>Os primeiros passos serão: remover todos os espaços das colunas com string, convertê-las para o tipo apropriado e remover excesso de NaNs.

## 3.1 Remover espaços & Corrigir tipos de dados e erros de escrita

In [ ]:
df_clear = DF_RAW.copy()
# Remover espaços em branco no início e no final das strings
df_clear = df_clear.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Converter para tipos númericos onde for necessario
df_clear["Delivery_person_Age"] = df_clear["Delivery_person_Age"].apply(pd.to_numeric, errors="coerce").astype("Int64")
df_clear["Delivery_person_Ratings"] = df_clear["Delivery_person_Ratings"].apply(pd.to_numeric, errors="coerce")
# Corrigir erro de escrita em colunas
df_clear.rename(columns={"Time_Orderd": "Time_Ordered"}, inplace=True)
# Corrigir erro de escrita em valores
df_clear["City"] = df_clear["City"].str.replace("Metropolitian", "Metropolitan", regex=False)
# Converter para tipo date.time para melhor manipulá-los
df_clear["Order_Date"] = pd.to_datetime(df_clear["Order_Date"], format="%d-%m-%Y").astype("string")
df_clear["Time_Ordered"] = pd.to_datetime(df_clear["Time_Ordered"], format="%H:%M:%S").dt.time.astype("string")
df_clear["Time_Order_picked"] = pd.to_datetime(df_clear["Time_Order_picked"], format="%H:%M:%S").dt.time.astype("string")
# Remover letras da coluna "Time_taken(min)" e converter tipo para "Int64"
df_clear["Time_taken(min)"] = df_clear["Time_taken(min)"].str.extract(r'(\d+)').astype("Int64")
# Remover excesso de palavras na coluna "Weatherconditions"
df_clear["Weatherconditions"] = df_clear["Weatherconditions"].str.replace("conditions ", "", regex=False)
# Mudar NaNs para "Unknown" em colunas que não guardam valores numéricos
cols = ["Weatherconditions", "Road_traffic_density", "City", "Festival"]
for col in cols:
    df_clear[col] = df_clear[col].str.replace("NaN", "Unknown", regex=False)

In [ ]:
print(f"Max rating: {df_clear["Delivery_person_Ratings"].max()}\nMin rating: {df_clear["Delivery_person_Ratings"].min()}")
df_clear[df_clear["Delivery_person_Ratings"] > 5.0]

Max rating: 6.0
Min rating: 1.0


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Ordered,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
3586,0x46d,BANGRES05DEL01,50,6.0,-12.970324,-77.645748,13.010324,77.685748,2022-03-13,<NA>,12:30:00,Unknown,Unknown,3,Meal,electric_scooter,0,No,Urban,25
4714,0x493,HYDRES17DEL01,50,6.0,-17.451976,-78.385883,17.561976,78.495883,2022-04-04,<NA>,23:20:00,Unknown,Unknown,3,Snack,bicycle,0,No,Metropolitan,27
5169,0x4f2,JAPRES08DEL01,50,6.0,-26.910262,-75.783013,27.020262,75.893013,2022-03-18,<NA>,18:50:00,Unknown,Unknown,3,Drinks,scooter,1,No,Metropolitan,20
5362,0x430,BANGRES19DEL01,50,6.0,12.914264,77.678400,13.024264,77.788400,2022-04-06,<NA>,20:55:00,Unknown,Unknown,3,Meal,electric_scooter,1,No,Metropolitan,18
5651,0xbef1,AGRRES13DEL02,50,6.0,-27.159795,-78.042990,27.209795,78.092990,2022-02-13,<NA>,18:05:00,Unknown,Unknown,3,Drinks,scooter,1,No,Metropolitan,20
6394,0x427,JAPRES06DEL02,50,6.0,26.911927,75.797282,27.041927,75.927282,2022-04-02,<NA>,22:10:00,Unknown,Unknown,3,Meal,electric_scooter,0,No,Urban,20
7031,0x3eb,MYSRES15DEL02,50,6.0,-12.352058,-76.606650,12.372058,76.626650,2022-03-15,<NA>,10:00:00,Unknown,Unknown,3,Buffet,scooter,1,No,Metropolitan,14
7681,0xd42,VADRES09DEL01,50,6.0,0.000000,0.000000,0.010000,0.010000,2022-03-21,<NA>,08:35:00,Unknown,Unknown,3,Meal,bicycle,1,No,Metropolitan,22
9499,0x3f0,BANGRES010DEL01,50,6.0,12.933298,77.614293,13.003298,77.684293,2022-03-12,<NA>,18:25:00,Unknown,Unknown,3,Drinks,scooter,1,No,Urban,17
9535,0x3ef,RANCHIRES13DEL01,50,6.0,-23.374989,-85.335486,23.444989,85.405486,2022-03-20,<NA>,17:55:00,Unknown,Unknown,3,Snack,bicycle,1,No,Metropolitan,32


Há valored de avaliação de 6,0. Como há poucos valores "errados" e, assumindo que o máximo é 5 "estrelas", eles serão corrigido para 5.

In [ ]:
df_clear.loc[df_clear["Delivery_person_Ratings"] > 5.0, "Delivery_person_Ratings"] = 5.0

## 3.2 Remover excesso de NaNs

In [ ]:
# Para manter a análise simples, linhas com mais de 2 NaNs serão removidas
df_clear = df_clear.dropna(thresh=len(df_clear.columns) - 2).reset_index(drop=True)

## 3.3 Corrigir e unir coordenadas em uma tupla em nova coluna

A muitas coordenadas com valores negativos, porem partindo do pressuposto de que todas as coordenadas são da India, não há coordendas negativas, portanto sera tomado como verdade que foram digitadas erroneamente e serão transformadas em positivas.<br>
Há também alguns valores de coordenadas (0, 0) até valores próximos. Este "local" é conhecido como "Ilha Nula" e é comumumente usada como um ponto de referência quando o GPS não consegue localizar um ponto, quando a coordenda digitada estava errada, entre outros casos. Portanto, estes valores serão mantidos com o pressuposto de que, mesmo que as coordenadas estejam erradas, a distancia entre o ponto inicial e o ponto final são reais.

In [ ]:
df_clear[(df_clear["Restaurant_latitude"] < 0) | (df_clear["Restaurant_longitude"] < 0) | (df_clear["Delivery_location_latitude"] < 0) | (df_clear["Delivery_location_longitude"] < 0)]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Ordered,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
90,0xc042,AGRRES010DEL01,34,4.7,-27.163303,78.057044,27.233303,78.127044,2022-02-16,23:00:00,23:05:00,Sandstorms,Low,1,Drinks,scooter,0,No,Metropolitan,15
276,0xc044,AGRRES12DEL03,32,4.7,-27.165108,78.015053,27.225108,78.075053,2022-02-13,20:55:00,21:10:00,Sandstorms,Jam,2,Meal,scooter,1,No,Metropolitan,31
1048,0xc086,GOARES010DEL03,20,4.7,-15.546594,73.760431,15.606594,73.820431,2022-02-13,20:00:00,20:10:00,Cloudy,Jam,2,Drinks,motorcycle,1,No,Metropolitan,29
1710,0xc067,BHPRES13DEL02,39,4.5,-23.230791,77.437020,23.250791,77.457020,2022-02-13,09:15:00,09:20:00,Cloudy,Low,0,Meal,motorcycle,1,No,Metropolitan,29
1898,0xc061,KOLRES16DEL01,25,4.0,-22.539129,88.365507,22.549129,88.375507,2022-02-15,09:25:00,09:30:00,Sandstorms,Low,0,Buffet,motorcycle,0,No,Metropolitan,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42276,0xc0b9,GOARES16DEL01,32,4.6,-15.498603,73.826911,15.568603,73.896911,2022-02-18,23:10:00,23:15:00,Sandstorms,Low,0,Snack,motorcycle,1,No,Metropolitan,24
42458,0xc06d,GOARES14DEL03,22,5.0,-15.157944,73.950889,15.217944,74.010889,2022-02-17,18:50:00,18:55:00,Sandstorms,Medium,1,Buffet,scooter,1,No,Metropolitan,27
43033,0xc094,KOCRES06DEL03,23,4.8,-9.982834,76.283268,10.072834,76.373268,2022-02-14,19:20:00,19:35:00,Sunny,Jam,0,Drinks,motorcycle,1,No,Metropolitan,21
43314,0xc0c2,AURGRES03DEL03,30,4.9,-19.874733,75.353942,19.904733,75.383942,2022-02-15,22:20:00,22:35:00,Fog,Low,1,Buffet,motorcycle,0,No,Metropolitan,15


In [ ]:
cols = ["Restaurant_latitude", "Restaurant_longitude", "Delivery_location_latitude", "Delivery_location_longitude"]
for col in cols:
    df_clear[col] = abs(df_clear[col])

Para melhor trabalhar com as coordenadas, elas serão armazenadas como uma tupla.


In [ ]:
df_clear["Restaurant_location"] = list(zip(df_clear["Restaurant_latitude"], df_clear["Restaurant_longitude"]))
df_clear["Delivery_location"] = list(zip(df_clear["Delivery_location_latitude"], df_clear["Delivery_location_longitude"]))
df_clear.drop(columns=["Restaurant_latitude", "Restaurant_longitude", "Delivery_location_latitude", "Delivery_location_longitude"], inplace=True)

## 3.4 Unir datas e horas

Para melhor trabalhar com o tempo, "Order_date" será unido com "Time_Ordered" e "Time_Order_picked". Pelo mesmo motivo, "Time_taken(min)" será transformado em um date.time baseado em "Time_Order_picked" + "Time_taken(min)".

In [ ]:
# Une as colunas de data e hora para uma melhor manipulação
df_clear["Time_Ordered"] = pd.to_datetime(df_clear["Order_Date"] + " " + df_clear["Time_Ordered"], format="%Y-%m-%d %H:%M:%S")
df_clear["Time_Order_picked"] = pd.to_datetime(df_clear['Order_Date'] + " " + df_clear["Time_Order_picked"], format="%Y-%m-%d %H:%M:%S")
# Adiciona 1 dia para pedidos feitos antes da meia noite e entregues depois da meia
df_clear["Time_Order_picked"] = df_clear.apply(lambda row: row["Time_Order_picked"] + pd.Timedelta(days=1) if row["Time_Ordered"] > row["Time_Order_picked"] else row["Time_Order_picked"], axis=1)
# Calcula a hora da entrega baseado em Time_order_pick + time_taken
df_clear["Time_Order_delivered"] = df_clear["Time_Order_picked"] + pd.to_timedelta(df_clear["Time_taken(min)"], unit='m')
# Remove as colunas originais de data e tempo
df_clear.drop(["Order_Date"], axis=1, inplace=True)

## 3.6 Preenchendo NaNs

In [ ]:
df_hot_encoded = pd.get_dummies(df_clear, columns=["Delivery_person_ID", ])

In [ ]:
# df2 = fill_nans_with_knn(df1, 'Delivery_person_Age', ['Delivery_person_ID', "Delivery_person_Ratings"])

In [ ]:
df_hot_encoded.shape

(43953, 1337)

In [ ]:
indexes = df_clear[df_clear['Delivery_person_Age'].isna()].index

In [ ]:
for index in indexes: print(df_clear.iloc[index]["Delivery_person_Age"])

<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>
<NA>


"Delivery_person_Ratings" pode ser preenchido com a média baseada no "Delivery_service_ID".

In [ ]:
df_aux = df_clear[["Delivery_person_ID", "Delivery_person_Ratings"]].dropna()
df_aux = df_aux.groupby(["Delivery_person_ID"]).mean().reset_index()
df_clear = df_clear.merge(df_aux, on="Delivery_person_ID", how="left", suffixes=('', '_mean'))
df_clear["Delivery_person_Ratings"] = df_clear["Delivery_person_Ratings"].fillna(df_clear["Delivery_person_Ratings_mean"])
df_clear.drop(columns=["Delivery_person_Ratings_mean"], inplace=True)

NaNs em "Time_ordered" podem ser preenchidos com base nos outros pedidos, agrupados pelo tipo de pedido.<br>A suposição aqui é que as refeições provavelmente têm um tempo de preparo mais longo do que o buffet, que tem um tempo mais longo do que as bebidas, que têm um tempo mais longo do que as lanches.

In [ ]:
df_aux = df_clear[["Time_Ordered", "Time_Order_picked", "Time_Order_delivered", "Type_of_order"]].dropna()
df_aux["Time_to_pick"] = (df_aux["Time_Order_picked"] - df_aux["Time_Ordered"]).dt.total_seconds() / 60
df_aux["Time_to_delivery"] = (df_aux["Time_Order_delivered"] - df_aux["Time_Order_picked"]).dt.total_seconds() / 60
df_aux = df_aux.drop(columns=["Time_Ordered", "Time_Order_picked", "Time_Order_delivered"])
df_times_grouped = df_aux.groupby(["Type_of_order"]).agg(["mean", "median"]).reset_index()
df_times_grouped

Type_of_order Time_to_pick        Time_to_delivery       
                        mean median             mean median
0        Buffet    10.026713   10.0        26.285925   26.0
1        Drinks     9.957787   10.0        26.198679   25.0
2          Meal     9.959158   10.0        26.417136   26.0
3         Snack    10.013975   10.0        26.272563   26.0

A suposição não foi exatamente verdadeira, mas a mediana de "Time_to_pick" pode ser usada para calcular quando os pedidos com NaNs foram feitos sem muita distorção nos dados.

In [ ]:
df_clear["Time_Ordered"] = df_clear["Time_Ordered"].fillna(df_clear["Time_Order_picked"] - pd.to_timedelta(df_times_grouped.loc[0, ("Time_to_pick", "median")], unit="m"))

## 3.7 Removendondo colunas inúteis

Em "multiple_delivery", não foi encontrado nenhum padrão que justificasse a sua existêcia, então a coluna será removida.

In [ ]:
df_clear.drop(columns=["multiple_deliveries"], inplace=True)

## 3.8 Reorganização e checagem dos dados

A ordem das colunas sera reorganizada para melhor vizualização.

In [ ]:
df_clear = df_clear[["ID", "Delivery_person_ID", "Delivery_person_Age", "Delivery_person_Ratings", "Type_of_order", "Time_Ordered", "Time_Order_picked", "Time_Order_delivered", "Time_taken(min)", "Type_of_vehicle", "Vehicle_condition", "City", "Road_traffic_density", "Weatherconditions", "Festival", "Restaurant_location", "Delivery_location"]]

In [ ]:
df_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43953 entries, 0 to 43952
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       43953 non-null  object        
 1   Delivery_person_ID       43953 non-null  object        
 2   Delivery_person_Age      43739 non-null  Int64         
 3   Delivery_person_Ratings  43953 non-null  float64       
 4   Type_of_order            43953 non-null  object        
 5   Time_Ordered             43953 non-null  datetime64[ns]
 6   Time_Order_picked        43953 non-null  datetime64[ns]
 7   Time_Order_delivered     43953 non-null  datetime64[ns]
 8   Time_taken(min)          43953 non-null  Int64         
 9   Type_of_vehicle          43953 non-null  object        
 10  Vehicle_condition        43953 non-null  int64         
 11  City                     43953 non-null  object        
 12  Road_traffic_density     43953 n

In [ ]:
plot_histogram(df_clear)

In [ ]:
check_outliers(df_clear)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Type_of_order,Time_Ordered,Time_Order_picked,Time_Order_delivered,Time_taken(min),Type_of_vehicle,Vehicle_condition,City,Road_traffic_density,Weatherconditions,Festival,Restaurant_location,Delivery_location
0,0x2876,RANCHIRES02DEL01,32,3.5,Snack,2022-03-08 21:35:00,2022-03-08 21:45:00,2022-03-08 22:20:00,35,scooter,1,Urban,Jam,Stormy,No,"(0.0, 0.0)","(0.11, 0.11)"
1,0x3f01,MUMRES15DEL01,39,3.8,Drinks,2022-03-16 19:55:00,2022-03-16 20:00:00,2022-03-16 20:37:00,37,motorcycle,1,Metropolitan,Jam,Windy,No,"(19.176269, 72.836721)","(19.286269, 72.946721)"
2,0xa053,JAPRES16DEL03,37,3.8,Snack,2022-04-04 20:45:00,2022-04-04 20:55:00,2022-04-04 21:37:00,42,motorcycle,1,Metropolitan,Jam,Sandstorms,No,"(26.849596, 75.800512)","(26.989596, 75.940512)"
3,0x7725,PUNERES12DEL01,38,3.7,Meal,2022-03-25 20:30:00,2022-03-25 20:40:00,2022-03-25 21:11:00,31,motorcycle,1,Metropolitan,Jam,Stormy,No,"(18.520016, 73.830547)","(18.630016, 73.940547)"
4,0x6edb,PUNERES06DEL01,34,3.8,Meal,2022-03-10 18:40:00,2022-03-10 18:55:00,2022-03-10 19:26:00,31,scooter,1,Metropolitan,Medium,Stormy,No,"(18.546258, 73.904337)","(18.656258, 74.014337)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,0x2ddd,MUMRES05DEL01,36,4.8,Buffet,2022-03-06 21:25:00,2022-03-06 21:40:00,2022-03-06 22:34:00,54,motorcycle,0,Metropolitan,Jam,Stormy,No,"(18.927584, 72.832585)","(18.997584, 72.902585)"
1407,0xd0f,SURRES02DEL02,31,4.0,Meal,2022-03-18 20:35:00,2022-03-18 20:40:00,2022-03-18 21:33:00,53,motorcycle,0,Urban,Jam,Stormy,Yes,"(21.186608, 72.794136)","(21.316608, 72.924136)"
1408,0xb839,MUMRES15DEL02,39,4.4,Meal,2022-04-02 21:00:00,2022-04-02 21:05:00,2022-04-02 21:59:00,54,motorcycle,0,Metropolitan,Jam,Sandstorms,No,"(19.176269, 72.836721)","(19.256269, 72.916721)"
1409,0x713d,JAPRES07DEL02,39,4.4,Drinks,2022-03-18 21:35:00,2022-03-18 21:45:00,2022-03-18 22:38:00,53,motorcycle,0,Metropolitan,Jam,Sandstorms,No,"(26.766536, 75.837333)","(26.896536, 75.967333)"


# 4.0 Análise descritiva

## 4.1 Primeiras KBQs
Por enquanto, as perguntas de négocio serão respondidas sem o uso de técnicas intermediárias/avançadas como groupby, agg, plotting, machine learning, etc...

### 1. Qual o número total de entregas cadastradas na base de dados?
**Resposta:** O número total de entregas cadastrados na base de dados é **43944**.

In [ ]:
len(df_clear)

43953

### 2. Quantos entregadores únicos existem na base de dados?
**Resposta:** Há **1320** IDs únicos na base de dados, porem aparentemente não são entregadores, mas sim empresas que oferecem o serviço de entrega, pois há multiplas idades para o mesmo ID de entregador.

Primeiramente foi notado nos datasets abaixo que há multiplas idades para o mesmo ID de entregador, o que sugere que provavelmente o ID não é do entregador, mas provavelmente da empresa que oferece o serviço de entrega.
Note que, apesar de não ser usado técnicas como groupby e agg, foi aberto uma exceção para o código abaixo, pois ele não visa responder a pergunta, mas sim apontar uma inconsistência na base de dados.

In [ ]:
df_aux = df_clear[["Delivery_person_ID", "Delivery_person_Age"]].dropna()
df_age_grouped = df_aux.groupby(["Delivery_person_ID"]).agg(["count"]).reset_index()
df_age_grouped

,Delivery_person_ID,Delivery_person_Age
,,count
0,AGRRES010DEL01,13
1,AGRRES010DEL02,14
2,AGRRES010DEL03,13
3,AGRRES01DEL01,9
4,AGRRES01DEL02,14
...,...,...
1315,VADRES19DEL02,58
1316,VADRES19DEL03,38
1317,VADRES20DEL01,56


In [ ]:
df_clear.loc[df_clear['Delivery_person_ID'] == 'VADRES19DEL02', ["Delivery_person_ID", "Delivery_person_Age"]]

,Delivery_person_ID,Delivery_person_Age
921,VADRES19DEL02,38
2160,VADRES19DEL02,33
2972,VADRES19DEL02,34
3091,VADRES19DEL02,28
3326,VADRES19DEL02,36
3390,VADRES19DEL02,28
4205,VADRES19DEL02,38
4899,VADRES19DEL02,24
4981,VADRES19DEL02,26
5393,VADRES19DEL02,26


Portanto será tratado como "serverviço de entrega único" e não "entregador único".

In [ ]:
df_clear.rename(columns={"Delivery_person_ID": "Delivery_service_ID"}, inplace=True)
len(df_clear["Delivery_service_ID"].unique())

1320

### 3. Qual a idade do entregador mais velho? E do mais novo?
**Resposta:** A maior idade é **50** anos e a menor idade é **15** anos.

In [ ]:
df_clear["Delivery_person_Age"].sort_values(ascending=True)

8811       15
36326      15
35767      15
37670      15
37677      15
         ... 
43359    <NA>
43534    <NA>
43743    <NA>
43871    <NA>
43887    <NA>
Name: Delivery_person_Age, Length: 43953, dtype: Int64

### 4. Qual o ID do entregador com a maior idade? E o ID do entregador com a menor idade?
**Resposta:** O ID do entregador mais velho é **"BANGRES05DEL01"** e o ID do entregador mais novo é **"JAPRES15DEL03"**.

In [ ]:
max_age_row = df_clear.loc[df_clear["Delivery_person_Age"].idxmax()]
min_age_row = df_clear.loc[df_clear["Delivery_person_Age"].idxmin()]
age_max_min = {
    "max": {
        "Delivery_service_ID": str(max_age_row["Delivery_service_ID"]),
        "Delivery_person_Age": int(max_age_row["Delivery_person_Age"])
    },
    "min": {
        "Delivery_service_ID": str(min_age_row["Delivery_service_ID"]),
        "Delivery_person_Age": int(min_age_row["Delivery_person_Age"])
    }
}
age_max_min


{'max': {'Delivery_service_ID': 'BANGRES05DEL01', 'Delivery_person_Age': 50},
 'min': {'Delivery_service_ID': 'JAPRES15DEL03', 'Delivery_person_Age': 15}}

### 5. Quais os nomes das condições climáticas?
**Resposta:** As condições climáticas são: 
- "Sunny"
- "Stormy"
- "Sandstorms"
- "Cloudy"
- "Fog"
- "Windy"
- "Unknown".

Lembrando que "Unknown" foi criado para conter os valores nulos da coluna.

In [ ]:
df_clear["Weatherconditions"].unique()

array(['Sunny', 'Stormy', 'Sandstorms', 'Cloudy', 'Fog', 'Windy',
       'Unknown'], dtype=object)

### 6. Quantas entregas foram realizadas sob condições climáticas de tempestade de areia (Sandstorms)?
**Resposta:** Foram feitas **7284** entregas com a condição "Sandstorms".

In [ ]:
len(df_clear.loc[df_clear["Weatherconditions"] == "Sandstorms", "Weatherconditions"])

7285

### 7. Quais eram as condicões climáticas da data mais recente de entrega?
**Resposta:** A condição climática da data mais recente de entrega foi **"Fog" (Neblina)**.

In [ ]:
df_clear.loc[df_clear["Time_Order_delivered"] == df_clear["Time_Order_delivered"].max(), "Weatherconditions"]

1288    Fog
Name: Weatherconditions, dtype: object

### 8. Quantos tipos de densidade de trânsito existem na base de dados? Quais os nomes delas?
**Resposta:** Existem **5** tipos de densidade de trânsito na base de dados:
- High
- Jam
- Low
- Medium
- Unknown

Lembrando que "Unknown" foi criado para conter os valores nulos da coluna.

In [ ]:
print(f"{len(df_clear["Road_traffic_density"].unique())} tipos de densidade.\n{df_clear["Road_traffic_density"].unique()}.")

5 tipos de densidade.
['High' 'Jam' 'Low' 'Medium' 'Unknown'].


### 9. Quantas entregas foram feitas em cada condição climática?
**Resposta:** A quantidade de entregas feitas em cada condição climática é: 
1. Fog: 7474
2. Stormy: 7400
3. Cloudy: 7338
4. Sandstorms: 7284
5. Windy: 7251
6. Sunny: 7106
7. Unknown: 91

In [ ]:
for condition in df_clear["Weatherconditions"].unique():
    print(f"{condition}: {len(df_clear.loc[df_clear["Weatherconditions"] == condition, "Weatherconditions"])}")

Sunny: 7109
Stormy: 7402
Sandstorms: 7285
Cloudy: 7339
Fog: 7476
Windy: 7251
Unknown: 91


### 10. Quantos serviços de entrega únicos fizeram entregas em cada condição climática?
**Resposta:** A quantidade de entregadores únicos que fizeram entregas em cada condição climática é: 
1. Fog: 1257
2. Cloudy: 1249
3. Sandstorms: 1242
4. Sunny: 1239
5. Stormy: 1237
6. Windy: 1233
7. Unknown: 87

In [ ]:
for condition in df_clear["Weatherconditions"].unique():
    print(f"{condition}: {df_clear.loc[df_clear["Weatherconditions"] == condition, "Delivery_service_ID"].nunique()}")

Sunny: 1239
Stormy: 1237
Sandstorms: 1242
Cloudy: 1249
Fog: 1257
Windy: 1233
Unknown: 87


### 11. Quantas entregas foram feitas em cada tipo de densidade de trânsito?
**Resposta:** A quantidade de entregas feitas em cada condição de transito é: 
1. Low: 15059
2. Jam: 13798
3. Medium: 10675
4. High: 4321
5. Unknown: 91

In [ ]:
for condition in df_clear["Road_traffic_density"].unique():
    print(f"{condition}: {len(df_clear.loc[df_clear["Road_traffic_density"] == condition, "Road_traffic_density"])}")

High: 4322
Jam: 13800
Low: 15062
Medium: 10678
Unknown: 91


### 12. Quantos entregadores únicos fizeram entregas em cada tipo de densidade de trânsito?
**Resposta:** A quantidade de entregadores únicos que fizeram entregas em cada densidade de trânsito é: 
1. Low: 1310
2. Jam: 1307
3. Medium: 1290
4. High: 794
5. Unknown: 87

In [ ]:
for condition in df_clear["Road_traffic_density"].unique():
    print(f"{condition}: {df_clear.loc[df_clear["Road_traffic_density"] == condition, "Delivery_service_ID"].nunique()}")

High: 794
Jam: 1307
Low: 1310
Medium: 1290
Unknown: 87


### 13. Quantos tipos de cidades únicas existem na base? Quais são os seus nomes?
***Resposta:*** Há **4** tipos de cidades únicas na base e seus nomes são: 
- Urban
- Metropolitian
- Semi-Urban
- Unknown

In [ ]:
print(f"Há {len(df_clear["City"].unique())} tipos de cidades únicas na base e seus nomes são: {df_clear["City"].unique()}.")

Há 4 tipos de cidades únicas na base e seus nomes são: ['Urban' 'Metropolitan' 'Semi-Urban' 'Unknown'].


### 14. Quantos tipos de veículos únicos existem na base? Quais são seus nomes?
***Resposta:*** Há **4** tipos de veículos únicos na base e seus nomes são:
- Bicycle
- scooter
- motorcycle
- electric_scooter

In [ ]:
print(f"Há {len(df_clear["Type_of_vehicle"].unique())} tipos de veículos únicos na base e seus nomes são: {df_clear["Type_of_vehicle"].unique()}.")

Há 4 tipos de veículos únicos na base e seus nomes são: ['motorcycle' 'scooter' 'electric_scooter' 'bicycle'].


### 15. Qual o tipo de veículo que mais fez entregas com a condição de trânsito pesado (High)?
***Resposta:*** O tipo de veículo que mais fez entregas com a condição de trânsito pesado foi **motorcycle**, com **2543** entregas.

In [ ]:
df_clear.loc[df_clear["Road_traffic_density"] == "High", "Type_of_vehicle"].value_counts()

Type_of_vehicle
motorcycle          2543
scooter             1439
electric_scooter     340
Name: count, dtype: int64

### 16. Qual o tipo de pedido mais feito durante condições climáticas de tempestade de areia?
***Resposta:*** O tipo de pedido mas feito durante a condição climáticas de tempestade de areia foi **meal**, com **1870** pedidos.

In [ ]:
df_clear.loc[df_clear["Weatherconditions"] == "Sandstorms", "Type_of_order"].value_counts()

Type_of_order
Meal      1871
Snack     1862
Drinks    1782
Buffet    1770
Name: count, dtype: int64

### 17. Qual o tipo de cidade com o maior número de pedidos de Bebidas (Drinks) feito em Scooter?
***Resposta:*** O tipo de cidade com o maior número de pedidos de bebidas feito em scooter foi **Metropolitan**, com **2718** pedidos.

In [ ]:
df_clear.loc[(df_clear["Type_of_order"] == "Drinks") & (df_clear["Type_of_vehicle"] == "scooter"), "City"].value_counts()

City
Metropolitan    2720
Urban            873
Unknown          102
Semi-Urban         1
Name: count, dtype: int64

### 18. Qual o tipo de cidade com o (os) entregador mais velho? E o nome da cidade com o (os) entregadores mais novos?
***Resposta:*** O tipo de cidade com o maior número de entregadores mais velhos e também mais novos é **Metropolitan**, seguido por **Urban**.

In [ ]:
df_clear.loc[df_clear["Delivery_person_Age"] == df_clear["Delivery_person_Age"].max()][["Delivery_person_Age", "City"]].groupby("City").count().reset_index().rename(columns={"Delivery_person_Age": "Count"})


,City,Count
0,Metropolitan,37
1,Unknown,2
2,Urban,14


Há 37 entregadores com a idade máxima do dataset no tipo de cidade Metropolitan.

In [ ]:
df_clear.loc[df_clear["Delivery_person_Age"] == df_clear["Delivery_person_Age"].min()][["Delivery_person_Age", "City"]].groupby("City").count().reset_index().rename(columns={"Delivery_person_Age": "Count"})

,City,Count
0,Metropolitan,27
1,Urban,11


Há 27 entregadores com a idade mínima do dataset no tipo de cidade Metropolitan.

### 19. Quantas entregas foram feitas durante o Festival?
***Resposta:*** Foram feitas **859** entregas durante o Festival.

In [ ]:
len(df_clear.loc[df_clear["Festival"] == "Yes", "ID"])

859

### 20. Quantos tipos de cidades únicas tiveram entregas feitas durante o Festival?
***Resposta:*** Houve **4** tipos de cidades únicas que tiveram entregas feitas durante o Festival.

In [ ]:
df_clear.loc[df_clear["Festival"] == "Yes", "City"].nunique()

4

### 21. Quantas entregas foram feitas durante o Festival no tipo de cidade Urban?
***Resposta:*** Foram feitas **109** entregas durante o Festival no tipo de cidade Urban.

In [ ]:
df_clear.loc[(df_clear["Festival"] == "Yes") & (df_clear["City"] == "Urban"), "ID"].count()

np.int64(109)

## 4.2 Próximas KBQs
As próximas perguntas de negócio serão respondidas usando o método SAPE e o uso de tecnicas intermediarias como groupby e agg.

### 1. Qual a média e mediana de idade dos entregadores por tipo de cidade?
**Resposta:** A média e mediana de idade dos entregadores por tipo de cidade é:
- Metropolitian: Média **29.78**, mediana **30**
- Urban: Média **28.88**, mediana **28**
- Semi-Urban: Média **32.5**, mediana **32**
- Unknown: Média **28.79**, mediana **28**

In [ ]:
# SAída: DataFrame com a média e mediana de idade dos entregadores agrupado por tipo de cidade
# Processo: Agrupar os dados por tipo de cidade e calcular a média e mediana da idade dos entregadores
# Entrada: DataFrame com os dados limpos
df_clear.groupby("City").agg({"Delivery_person_Age": ["mean", "median"]})

Delivery_person_Age       
                            mean median
City                                   
Metropolitan           29.785124   30.0
Semi-Urban             32.565789   32.0
Unknown                28.789982   28.0
Urban                  28.880115   28.0

### 2. Qual a média das avaliações das entregas feitas por densidade de tráfego?
**Resposta:** A média das avaliações das entregas feitas por densidade de tráfego é:
- Low: 4.64
- Medium: 4.66
- High: 4.65
- Jam: 4.59
- Unknown: 3.32

In [ ]:
# SAída: DataFrame com a média das avaliações das entregas feitas agrupadas por densidade de tráfego
# Processo: Agrupar os dados por densidade de tráfego e calcular a média das avaliações das entregas
# Entrada: DataFrame com os dados limpos
df_clear.groupby("Road_traffic_density").agg({"Delivery_person_Ratings": "mean"})

,Delivery_person_Ratings
Road_traffic_density,
High,4.654529
Jam,4.596266
Low,4.647497
Medium,4.660708
Unknown,3.329670


### 3. Qual a média, mediana e desvio padrão do tempo das entregas feitas agrupadas por tipo de cidade e tipo de veículo?
**Resposta:** Agrupados por tipo de cidade e tipo de veículo, a média, mediana e desvio padrão das entregas feitas é:
- Metropolitan
    - Bicyle: média **25.4**, mediana **25**, desvio padrão **7.46**
    - Electric Scooter: média **25.38**, mediana **25**, desvio padrão **8.5**
    - Motorcycle: média **28.49**, mediana **27**, desvio padrão **9.38**
    - Scooter: média **25.64**, mediana **25**, desvio padrão **8.56**
- Urban
    - Scooter: média **21.32**, mediana **19**, desvio padrão **8.17**
    - Electric Scooter: média **21.5**, mediana **20**, desvio padrão **7.89**
    - Motorcycle: média **24.31**, mediana **23**, desvio padrão **9.21**
- Semi-Urban
    - Electric Scooter: média **49.00**, mediana **49.00**, desvio padrão **0**
    - Scooter: média **47.4**, mediana **48**, desvio padrão **1.5**
    - Motorcycle: média **49.91**, mediana **49**, desvio padrão **2.75**
- Unknown
    - Electric Scooter: média **21.47**, mediana **19**, desvio padrão **8.1**
    - Mortorcycle: média **23.22**, mediana **22**, desvio padrão **8.64**
    - Scooter: média **20.61**, mediana **19**, desvio padrão **7.79**

*OBS:*<br>
&nbsp;&nbsp;&nbsp;&nbsp; - *Valores representam minutos, por exemplo, 1.5 representa 1 minuto e 30 segundos.*<br>

In [ ]:
# SAída: DataFrame com a média, mediana e desvio padrão das entregas feitas agrupadas por tipo de cidade e tipo de veículo
# Processo: Agrupar as entregas por tipo de cidade e tipo de veículo e calcular a média, mediana e desvio padrão
# Entrada: DataFrame com os dados limpos
df = df_clear[["City", "Type_of_vehicle"]].copy()
df["time_diff(min)"] = (df_clear["Time_Order_delivered"] - df_clear["Time_Order_picked"]).dt.total_seconds() / 60
df.groupby(["City", "Type_of_vehicle"]).agg({"time_diff(min)": ["mean", "median", "std"]})


time_diff(min)                 
                                        mean median       std
City         Type_of_vehicle                                 
Metropolitan bicycle               25.400000   25.0  7.462286
             electric_scooter      25.376697   25.0  8.506520
             motorcycle            28.491981   27.0  9.388846
             scooter               25.641135   25.0  8.566167
Semi-Urban   electric_scooter      49.000000   49.0  0.000000
             motorcycle            49.916084   49.0  2.751431
             scooter               47.400000   48.0  1.505545
Unknown      electric_scooter      21.471910   19.0  8.101753
             motorcycle            23.226601   22.0  8.642833
             scooter               20.616071   19.0  7.799209
Urban        electric_scooter      21.493947   20.0  7.889577
             motorcycle            24.312971   23.0  9.213521
             scooter               21.320658   19.0  8.175336

### 4. Qual a primeira e a última data de entrega por cada tipo de cidade?
**Resposta:** A primeira e última data de entrega por cada tipo de cidade são as seguintes:	
- Metropolitan: primeira em *2022-02-11 00:17*, e última em *2022-04-07 00:42*
- Semi-Urban: primeira em *2022-02-11 22:52*, e última em *2022-04-06 22:41*
- Unknown: primeira em *2022-02-11 8:35*, e última em	*2022-04-07 00:48*
- Urban: primeira em *2022-02-11 00:21*, e última em *2022-04-07 00:31*

In [ ]:
# SAída: DataFrame com a primeira e a última data de entrega por cada tipo de cidade
# Processo: Agrupar as entregas por tipo de cidade e recuperar a primeira e a última data de entrega
# Entrada: DataFrame com os dados limpos
df_clear.groupby("City")["Time_Order_delivered"].agg(["min", "max"])

,min,max
City,,
Metropolitan,2022-02-11 00:17:00,2022-04-07 00:42:00
Semi-Urban,2022-02-11 22:52:00,2022-04-06 22:41:00
Unknown,2022-02-11 08:35:00,2022-04-07 00:48:00
Urban,2022-02-11 00:21:00,2022-04-07 00:31:00


### 5. Qual a média de avaliações das entregas feitas por condições climáticas?
**Resposta:** A média de avaliações das entregas feitas agrupadas por condições climáticas é:
1. Sunny:         4.66
2. Fog:           4.65
3. Cloudy:        4.65
4. Windy:         4.62
5. Stormy:        4.61
6. Sandstorms:    4.61
7. Unknown:       3.33

In [ ]:
# SAída: Média de avaliações das entregas feitas por condições climáticas
# Processo: Agrupar as entregas por condições climáticas e calcular a média das avaliações
# Entrada: DataFrame com os dados limpos
df_clear.groupby("Weatherconditions")["Delivery_person_Ratings"].mean()

Weatherconditions
Cloudy        4.652476
Fog           4.653986
Sandstorms    4.613917
Stormy        4.614509
Sunny         4.659179
Unknown       3.329670
Windy         4.617872
Name: Delivery_person_Ratings, dtype: float64

### 6. Qual o valor da avaliação mais baixa feita por tipo de condição climática e densidade de tráfego?
**Resposta:** Os valores das avaliações mais baixas feitas agrupadas por tipo de condição climática e densidade de tráfego são as seguintes:
- Cloudy: 
    - High: 4.0
    - Jam: 4.0
    - Low: 3.5
    - Medium: 4.0
- Fog:
    - High: 4.0
    - Jam: 4.0
    - Low: 3.5
    - Medium: 4.0
- Sandstorms:
    - High: 4.0
    - Jam: 3.5
    - Low: 4.0
    - Medium: 3.5
- Stormy:
    - High: 4.0
    - Jam: 3.5
    - Low: 4.0
    - Medium: 3.5
- Sunny:
    - High: 3.5
    - Jam: 3.5
    - Low: 2.5
    - Medium: 4.0
- Windy:
    - High: 4.0
    - Jam: 3.5
    - Low: 4.0
    - Medium: 3.5
- Unknown:
    - Unknown: 1.0

In [ ]:
# SAída: Valor da avaliação mais baixa agrupada por tipo de condição climática e densidade de tráfego
# Processo: Agrupar as entregas por tipo de condição climática e densidade de tráfego e calcular o valor mínimo das avaliações
# Entrada: DataFrame com os dados limpos
df_clear.groupby(["Weatherconditions", "Road_traffic_density"])["Delivery_person_Ratings"].agg(["min"])

min
Weatherconditions Road_traffic_density     
Cloudy            High                  4.0
                  Jam                   4.0
                  Low                   3.5
                  Medium                4.0
Fog               High                  4.0
                  Jam                   4.0
                  Low                   3.5
                  Medium                4.0
Sandstorms        High                  4.0
                  Jam                   3.5
                  Low                   4.0
                  Medium                3.5
Stormy            High                  4.0
                  Jam                   3.5
                  Low                   4.0
                  Medium                3.5
Sunny             High                  3.5
                  Jam                   3.5
                  Low                   2.5
                  Medium                4.0
Unknown           Unknown               1.0
Windy             High                  4.0
                  Jam                   3.5
                  Low                   4.0
                  Medium                3.5

### 7. Qual a média, mediana e desvio padrão das avaliações por cada tipo de condições de veículos?
**Resposta:** A média, mediana e desvio padrão das avaliações por cada tipo de condições de veículos são as seguintes:
- 0:
    - média: **4.60**
    - mediana: **4.7**
    - desvio padrão: **0.29**
- 1:
    - média: **4.64**
    - mediana: **4.7**
    - desvio padrão: **0.31**
- 2:
    - média: **4.64**
    - mediana: **4.7**
    - desvio padrão: **0.32**
- 3:
    - média: **3.32**
    - mediana:	**5.00**
    - desvio padrão:	**1.98**

In [ ]:
# SAída: DataFrame com a média, mediana e desvio padrão das avaliações por cada tipo de condições de veículos
# Processo: Agrupar as entregas por tipo de condições de veículos e calcular a média, mediana e desvio padrão das avaliações
# Entrada: DataFrame com os dados limpos
df_clear.groupby("Vehicle_condition")["Delivery_person_Ratings"].agg(["mean", "median", "std"])

,mean,median,std
Vehicle_condition,,,
0,4.609545,4.7,0.296864
1,4.649306,4.7,0.318644
2,4.646968,4.7,0.321044
3,3.329670,5.0,1.983571


### 8. Qual a avaliação média das entregas feitas durante o Festival?
**Resposta:** A média das das avaliações das entregas feitas durante o Festival é **4.49**.

In [ ]:
# SAída: Avaliação média das entregas feitas durante o Festival
# Processo: Filtrar as entregas feitas durante o Festival e calcular a média das avaliações
# Entrada: DataFrame com os dados limpos
df_clear[df_clear["Festival"] == "Yes"]["Delivery_person_Ratings"].mean()

np.float64(4.492196397951636)

### 9. Qual a menor avaliação feita em uma entrega durante o Festival por cidade?
**Resposta:** As menores avaliações feitas em uma entrega durante o Festival agrupada por cidade são as seguintes:
- Metropolitan: **1.0**
- Semi-Urban: **3.5**
- Urban: **2.5**
- Unknown: **4.0**

In [ ]:
# SAída: Menor avaliação feita em uma entrega durante o Festival por cidade
# Processo: Filtrar as entregas feitas durante o Festival e agrupar por cidade para encontrar a menor avaliação
# Entrada: DataFrame com os dados limpos
df_clear[df_clear["Festival"] == "Yes"].groupby("City")["Delivery_person_Ratings"].min()

City
Metropolitan    1.0
Semi-Urban      3.5
Unknown         4.0
Urban           2.5
Name: Delivery_person_Ratings, dtype: float64

### 10. Qual a maior avaliação feita por tipo de pedido?
**Resposta:** As maiores avaliações agrupadas por tipo de pedido são:
- Buffet: **5.0**
- Drinks: **5.0**
- Meal: **5.0**
- Snack: **5.0**

In [ ]:
# SAída: Maior avaliação feita por tipo de pedido
# Processo: Agrupar por tipo de pedido e encontrar a maior avaliação
# Entrada: DataFrame com os dados limpos
df_clear.groupby("Type_of_order")["Delivery_person_Ratings"].max()

Type_of_order
Buffet    5.0
Drinks    5.0
Meal      5.0
Snack     5.0
Name: Delivery_person_Ratings, dtype: float64

## 4.3 Visão da empresa
Visualização de métricas da empresa.

### 1. Quantidades de pedido por dia.

In [ ]:
# Agrupar pedidos por dias
df_clear["Day_Ordered"] = pd.to_datetime(df_clear["Time_Ordered"]).dt.date
df = df_clear[["ID", "Day_Ordered"]].groupby("Day_Ordered").count().reset_index()
df = df[["Day_Ordered", "ID"]].pivot_table(index="Day_Ordered", aggfunc="sum").reset_index()

In [ ]:
# Desenhar gráfico de barras
fig = px.bar(df, x="Day_Ordered", y="ID", title="Quantidade de pedidos por dia", labels={"Day_Ordered": "Data", "ID": "Quantidade de pedidos"})
fig.update_layout(title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    xaxis={"title_font": {"size": 18}, "showgrid": True},
    yaxis={"title_font": {"size": 18}, "showgrid": True})
fig.show()

### 2. Quantidade de pedidos por semana.

In [ ]:
# Agrupar dias em semanas do ano
df_clear["Week_Ordered"] = pd.to_datetime(df_clear["Day_Ordered"]).dt.strftime("%W").astype(int)
df = df_clear[["Week_Ordered", "ID"]].pivot_table(index="Week_Ordered", aggfunc="count").reset_index()
# Criar um DataFrame com todas as semanas do periodo
all_weeks = pd.DataFrame({"Week_Ordered": range(6, 15)})
# Mesclar com o DataFrame original
df = pd.merge(all_weeks, df, on="Week_Ordered", how="left").fillna(0)

In [ ]:
# Desenhar gráfico de linha
fig = px.line(df, x="Week_Ordered", y="ID", title="Quantidade de pedidos por semana", labels={"Week_Ordered": "Semana do ano", "ID": "Quantidade de pedidos"}, markers=True)
fig.update_layout(title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    xaxis={"title_font": {"size": 18}, "showgrid": True},
    yaxis={"title_font": {"size": 18}, "showgrid": True})
fig.show()

In [ ]:
metric01 = df["ID"].sum()
metric02 = df["ID"].max()
metric03 = df["ID"].min()
metric04 = df["ID"].mean()
metric05 = df["ID"].std()
metric06 = df["ID"].diff().mean()
metric06

np.float64(30.375)

### 3. Distribuição dos pedidos por densidade de tráfego

In [ ]:
# Agrupar por densidade de tráfego
df = df_clear[["ID", "Road_traffic_density"]].groupby("Road_traffic_density").count().reset_index()
# Calcular porcentagem
df["percent"] = df["ID"] / df["ID"].sum()

In [ ]:
# Desenhar gráfico de pizza
fig = px.pie(df, values="percent", names="Road_traffic_density", title="Distribuição de pedidos por densidade de tráfego")
fig.update_layout(width=700, height=600,
    title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    legend={"font": {"size": 18}})
fig.show()

### 4. Comparação de volume de pedidos por tipo de cidade e densidade de tráfego

In [ ]:
# Agrupar por cidade e densidade de tráfego
df = df_clear[["ID", "City", "Road_traffic_density"]].groupby(["City", "Road_traffic_density"]).count().reset_index()

In [ ]:
# Criar gráfico de barras empilhadas
fig = px.bar(df, x="City", y="ID", color="Road_traffic_density", title="Pedidos por tipo de cidade separados por densidade de tráfego", labels={"City": "Tipo de cidade", "ID": "Quantidade de pedidos", "Road_traffic_density": "Densidade de tráfego"}, barmode="stack", log_y=True)
fig.update_layout(title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    xaxis={"title_font": {"size": 18}, "showgrid": True},
    yaxis={"title_font": {"size": 18}, "showgrid": True})
fig.show()

### 5. Quantidade de pedidos por entregador agrupado por semana

In [ ]:
# Agrupar por semana e Delivery_service_ID e contar a quantidade de IDs
df_aux = df_clear.groupby(["Week_Ordered", "Delivery_service_ID"]).size().reset_index(name="ID_Count")
# Agrupar novamente por semana e calcular a soma total de IDs e a quantidade de Delivery_person_ID únicos
df_aux = df_aux.groupby("Week_Ordered").agg({"ID_Count": "sum", "Delivery_service_ID": "nunique"}).reset_index()
# Calcular a média de IDs por Delivery_service_ID para cada semana
df_aux["Order_by_delivery"] = df_aux["ID_Count"] / df_aux["Delivery_service_ID"]
# Mesclar com o DataFrame das semanas completas
df_aux = pd.merge(all_weeks, df_aux, on="Week_Ordered", how="left").fillna(0)

In [ ]:
df_aux["ID_Count"].sum()

np.float64(43953.0)

In [ ]:
df_aux

,Week_Ordered,ID_Count,Delivery_service_ID,Order_by_delivery
0,6,2701.0,600.0,4.501667
1,7,4281.0,600.0,7.135000
2,8,0.0,0.0,0.000000
3,9,6200.0,720.0,8.611111
4,10,7271.0,720.0,10.098611
5,11,7129.0,720.0,9.901389
6,12,6160.0,720.0,8.555556
7,13,7267.0,720.0,10.093056
8,14,2944.0,720.0,4.088889


In [ ]:
# Criar gráfico de linhas
fig = px.line(df_aux, x="Week_Ordered", y="Order_by_delivery", title="Quantidade de pedidos por entregador agrupado por semana", labels={"Week_Ordered": "Semana", "Order_by_delivery": "Quantidade de pedidos por entregador", "ID_Count": "Quantidade de pedidos", "Delivery_service_ID": "Quantidade de entregadores"}, markers=True, hover_data=["ID_Count", "Delivery_service_ID"])
fig.update_layout(title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    xaxis={"title_font": {"size": 18}, "showgrid": True},
    yaxis={"title_font": {"size": 18}, "showgrid": True})
fig.show()

### 6 Localização central de entregas de cada tipo de cidade agrupado por tipo de tráfego

In [ ]:
df_aux = df_clear[["City", "Road_traffic_density", "Delivery_location"]].copy()
df_aux[["Delivery_location_x", "Delivery_location_y"]] = pd.DataFrame(df_aux["Delivery_location"].tolist(), index=df_aux.index)
df_aux = df_aux.drop(columns=["Delivery_location"])
df_aux = df_aux.groupby(["City", "Road_traffic_density"]).median().reset_index()
df_aux["Delivery_location"] = list(zip(df_aux["Delivery_location_x"], df_aux["Delivery_location_y"]))
df_aux = df_aux.drop(columns=["Delivery_location_x", "Delivery_location_y"])
fig = folium.Map()
title_html = """
            <h3 align="center" style="font-size:20px"><b>Localização central de entregas agrupados por tipo de cidade e tráfego</b></h3>
            """
fig.get_root().html.add_child(folium.Element(title_html))
for index, location_info in df_aux.iterrows():
    popup_html = f"""
    <div style="max-width: 150px">
        {location_info['Delivery_location'][0]}° N, {location_info['Delivery_location'][1]}° W<br>
        City type: {location_info['City']}<br>
        Traffic: {location_info['Road_traffic_density']}
    </div>
    """
    folium.Marker(location_info["Delivery_location"], popup=folium.Popup(popup_html, max_width=350)).add_to(fig)
fig.fit_bounds(df_aux["Delivery_location"].tolist())
fig

### 7. Quantidade de restaurantes

In [ ]:
fig = folium.Map(location=(21.904992, 79.417227), zoom_start=5)
df_aux = df_clear[["ID", "Restaurant_location"]].groupby("Restaurant_location").count().reset_index()
df_aux.drop(index=0, inplace=True)
for index, location_info in df_aux.iterrows():
    popup_html = f"""
    <div style="max-width: 150px">
        {location_info['Restaurant_location'][0]}° N, {location_info['Restaurant_location'][1]}° W<br>
        Number of deliveries: {location_info['ID']}
    </div>
    """
    folium.Marker(location_info["Restaurant_location"], popup=folium.Popup(popup_html, max_width=350)).add_to(fig)
fig

## 4.4 Visão dos entregadores
Visualização de métricas dos entregadores.

### 1. Menor e maior idade dos entregadores.

Aqui foi assumido como premissa que um "Delivery_service_ID" não possui entregadores com idades iguais, visto que os IDs não são únicos.

In [ ]:
df_aux = df_clear[["Delivery_service_ID", "Delivery_person_Age"]].groupby("Delivery_person_Age").nunique().reset_index().sort_values(by="Delivery_person_Age")
fig = px.bar(df_aux, x="Delivery_person_Age", y="Delivery_service_ID", title="Quantidade de entregadores por idade", labels={"Delivery_person_Age": "Idade", "Delivery_service_ID": "Quantidade"})
fig.update_layout(title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    xaxis={"title_font": {"size": 18}, "showgrid": True},
    yaxis={"title_font": {"size": 18}, "showgrid": True})
fig.show()

In [ ]:
print(f"A menor idade é {df_clear['Delivery_person_Age'].min()} e a maior idade é {df_clear['Delivery_person_Age'].max()} anos.")

A menor idade é 15 e a maior idade é 50 anos.


### 2. Melhor e pior condição de veículos.

Aqui foi assumido como premissa que um "Delivery_service_ID" não possui veículos com condições iguais, visto que não um identificador para veículos.

In [ ]:
df_aux = df_clear[["Delivery_service_ID", "Vehicle_condition"]].groupby("Vehicle_condition").nunique().reset_index().sort_values(by="Vehicle_condition")
x_values = df_aux["Vehicle_condition"].unique()
fig = px.bar(df_aux, x="Vehicle_condition", y="Delivery_service_ID", title="Quantidade de veículos por condição", labels={"Vehicle_condition": "Condição", "Delivery_service_ID": "Quantidade"})
fig.update_layout(title={"x": 0.5, "xanchor": "center", "font": {"size": 24}},
    xaxis={"title_font": {"size": 18}, "showgrid": True, "tickmode": "array", "tickvals": x_values, "ticktext": ["muito ruim", "ruim", "bom", "muito bom"]},
    yaxis={"title_font": {"size": 18}, "showgrid": True, "type": "log"})
fig.show()

In [ ]:
print(f'A melhor condição é {df_aux['Vehicle_condition'].max()} e a pior condição é {df_aux['Vehicle_condition'].min()}, numa escala onde 0 = "muito ruim" e 3 = "muito bom".')

A melhor condição é 3 e a pior condição é 0, numa escala onde 0 = "muito ruim" e 3 = "muito bom".


### 3. Avaliação média por entregador.

In [ ]:
delivery_rating_by_delivery_person = df_clear[["Delivery_service_ID", "Delivery_person_Ratings"]].groupby("Delivery_service_ID").mean().reset_index().sort_values(by="Delivery_person_Ratings", ascending=False).rename(columns={"Delivery_person_Ratings": "Mean_rating"})
delivery_rating_by_delivery_person["Mean_rating"] = delivery_rating_by_delivery_person["Mean_rating"].round(2)
delivery_rating_by_delivery_person

,Delivery_service_ID,Mean_rating
158,AURGRES13DEL03,4.90
816,KOCRES13DEL01,4.86
497,GOARES05DEL03,4.84
959,LUDHRES20DEL03,4.84
55,AGRRES19DEL02,4.84
...,...,...
65,ALHRES01DEL03,4.40
281,BHPRES14DEL03,4.39
847,KOLRES02DEL02,4.34
777,KNPRES20DEL01,4.31


### 4. Avaliação média e desvio padrão por tipo de tráfego.

In [ ]:
delivery_rating_by_road_traffic = df_clear[["Delivery_person_Ratings", "Road_traffic_density"]].groupby("Road_traffic_density").agg({"Delivery_person_Ratings": ["mean", "std"]})
delivery_rating_by_road_traffic.columns = ["Mean_Rating", "Std_Rating"]
delivery_rating_by_road_traffic.reset_index(inplace=True)
delivery_rating_by_road_traffic


,Road_traffic_density,Mean_Rating,Std_Rating
0,High,4.654529,0.270823
1,Jam,4.596266,0.327438
2,Low,4.647497,0.333140
3,Medium,4.660708,0.272838
4,Unknown,3.329670,1.983571


### 5. Avaliação média e desvio padrão por condição climática.

In [ ]:
delivery_rating_by_weatherconditions = df_clear[["Delivery_person_Ratings", "Weatherconditions"]].groupby("Weatherconditions").agg({"Delivery_person_Ratings": ["mean", "std"]})
delivery_rating_by_weatherconditions.columns = ["Mean_Rating", "Std_Rating"]
delivery_rating_by_weatherconditions.reset_index(inplace=True)
delivery_rating_by_weatherconditions

,Weatherconditions,Mean_Rating,Std_Rating
0,Cloudy,4.652476,0.278828
1,Fog,4.653986,0.272614
2,Sandstorms,4.613917,0.308219
3,Stormy,4.614509,0.310369
4,Sunny,4.659179,0.390097
5,Unknown,3.329670,1.983571
6,Windy,4.617872,0.302701


### 6. Top 10 entregadores mais rápidos por tipo de cidade.

In [ ]:
df_clear["Distance(km)"] = df_clear.apply(lambda x: haversine(x["Restaurant_location"], x["Delivery_location"]), axis=1)
df_clear["Velocity(km/h)"] = df_clear["Distance(km)"] / (df_clear["Time_taken(min)"] / 60)

PS: Os resultados aqui são estranhos, principalmente no tipo de cidade "Unknown", que tem uma scooter eletrica que faz >100km/h, por exemplo.

In [ ]:
fastest_delivery_by_city = df_clear[["City", "Delivery_service_ID", "Velocity(km/h)"]].groupby(["City", "Delivery_service_ID"]).mean().sort_values(["City", "Velocity(km/h)"], ascending=False).reset_index()
fastest_delivery_by_city = fastest_delivery_by_city.groupby("City").head(10).reset_index(drop=True)
fastest_delivery_by_city

,City,Delivery_service_ID,Velocity(km/h)
0,Urban,AGRRES20DEL02,89.279343
1,Urban,KOCRES04DEL02,84.271716
2,Urban,ALHRES20DEL03,74.007102
3,Urban,DEHRES03DEL02,73.176129
4,Urban,AURGRES05DEL02,72.422923
5,Urban,ALHRES12DEL03,71.486994
6,Urban,GOARES11DEL02,70.833599
7,Urban,ALHRES07DEL03,66.402068
8,Urban,BHPRES20DEL02,65.423555
9,Urban,KOCRES03DEL02,64.782558


### 7. Top 10 entregadores mais lentos por tipo de cidade.

In [ ]:
slowest_delivery_by_city = df_clear[["City", "Delivery_service_ID", "Velocity(km/h)"]].groupby(["City", "Delivery_service_ID"]).mean().sort_values(["City", "Velocity(km/h)"]).reset_index()
slowest_delivery_by_city = slowest_delivery_by_city.groupby("City").head(10).reset_index(drop=True)
slowest_delivery_by_city

,City,Delivery_service_ID,Velocity(km/h)
0,Metropolitan,KOLRES09DEL01,9.463417
1,Metropolitan,GOARES19DEL01,10.164947
2,Metropolitan,LUDHRES20DEL01,11.324185
3,Metropolitan,AURGRES13DEL01,12.425341
4,Metropolitan,GOARES06DEL01,12.820195
5,Metropolitan,AGRRES17DEL01,12.869806
6,Metropolitan,AURGRES15DEL01,13.230448
7,Metropolitan,AURGRES13DEL03,13.438349
8,Metropolitan,GOARES07DEL03,13.720587
9,Metropolitan,KOCRES20DEL01,13.786311


## 4.5 Visão dos restaurantes
Visualização de métricas dos restaurantes.

### 1. Quantidade de serviços de entrega únicos.

In [ ]:
print(f"O número de entregadores únicos é {df_clear["Delivery_service_ID"].nunique()}.")

O número de entregadores únicos é 1320.


### 2. Distância média entre restaurantes e locais de entrega.

In [ ]:
mean_distance = df_clear.apply(lambda x: haversine(x["Restaurant_location"], x["Delivery_location"]), axis=1).mean()
print(f"A distância média entre restaurantes e locais de entrega é {mean_distance:.2f} km.")

A distância média entre restaurantes e locais de entrega é 9.73 km.


### 3. Tempo médio e desvio padrão do pedido feito até a coleta, agrupado por tipo de cidade.

In [ ]:
df_clear["Prepare_time(min)"] = ((df_clear["Time_Order_picked"] - df_clear["Time_Ordered"]).dt.total_seconds() / 60).astype(int)
df_aux = df_clear[["Prepare_time(min)", "City"]].groupby("City").agg(["mean", "std"]).reset_index()
df_aux.columns = ["City", "Mean_time", "Std_time"]
df_aux.reset_index(drop=True)

,City,Mean_time,Std_time
0,Metropolitan,9.974898,4.082499
1,Semi-Urban,10.160256,4.072721
2,Unknown,10.095986,4.083136
3,Urban,10.022994,4.086276


### 4. Tempo médio e desvio padrão do pedido feito até a coleta, agrupado por tipo de cidade e tipo de pedido.

In [ ]:
df_aux = df_clear[["Prepare_time(min)", "City", "Type_of_order"]].groupby(["City", "Type_of_order"]).agg(["mean", "std"]).reset_index()
df_aux.columns = ["City", "Type_of_order", "Mean_time", "Std_time"]
df_aux.reset_index(drop=True)

,City,Type_of_order,Mean_time,Std_time
0,Metropolitan,Buffet,10.009862,4.110561
1,Metropolitan,Drinks,9.958830,4.077380
2,Metropolitan,Meal,9.943134,4.067763
3,Metropolitan,Snack,9.988504,4.075173
4,Semi-Urban,Buffet,9.880952,4.055724
5,Semi-Urban,Drinks,10.156250,4.112211
6,Semi-Urban,Meal,10.967742,4.362598
7,Semi-Urban,Snack,9.901961,3.935759
8,Unknown,Buffet,10.119454,3.956449
9,Unknown,Drinks,10.311355,3.998407


### 5. Tempo médio e desvio padrão do pedido feito até a coleta, por tipo de cidade e tipo de tráfego.

In [ ]:
df_aux = df_clear[["Prepare_time(min)", "City", "Road_traffic_density"]].groupby(["City", "Road_traffic_density"]).agg(["mean", "std"]).reset_index()
df_aux.columns = ["City", "Road_traffic_density", "Mean_time", "Std_time"]
df_aux.reset_index(drop=True)

,City,Road_traffic_density,Mean_time,Std_time
0,Metropolitan,High,9.939247,4.093181
1,Metropolitan,Jam,9.954260,4.073445
2,Metropolitan,Low,9.978236,4.100735
3,Metropolitan,Medium,10.012315,4.082967
4,Metropolitan,Unknown,10.000000,0.000000
5,Semi-Urban,High,8.750000,3.872983
6,Semi-Urban,Jam,10.346154,4.043775
7,Semi-Urban,Medium,10.000000,4.714045
8,Unknown,High,9.947917,4.135520
9,Unknown,Jam,10.265957,4.091939


### 6. Tempo médio do pedido feito até a coleta durante O Festival.

In [ ]:
df_aux = df_clear[df_clear["Festival"] == "Yes"][["Prepare_time(min)", "Festival"]].groupby("Festival").agg(["mean", "std"]).reset_index()
df_aux.columns = ["Festival", "Mean_time", "Std_time"]
df_aux.reset_index(drop=True)

,Festival,Mean_time,Std_time
0,Yes,9.877765,4.112657
